In [ ]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.utils import get_file, to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import numpy as np
import random
import sys
import io

from module.conf import PROJECT_DIR

In [ ]:
# import os
# try:
#     # Disable all GPUS
#     tf.config.set_visible_devices([], 'GPU')
#     visible_devices = tf.config.get_visible_devices()
#     for device in visible_devices:
#         assert device.device_type != 'GPU'
# except:
#     # Invalid device or cannot modify virtual devices once initialized.
#     pass
strategy  = tf.distribute.get_strategy()
# with strategy.scope():
with tf.device("/cpu:0"):
    print(f"{tf.config.list_physical_devices() }")
# print(f"{tf.config.list_physical_devices('GPU') }")

In [ ]:
from transformers import AutoModel, AutoTokenizer, PreTrainedTokenizer, BertModel
phoBERT: BertModel = AutoModel.from_pretrained(PROJECT_DIR + "/models/phobert-base")
custokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(PROJECT_DIR + "/models/phobert-base", use_fast=False)

In [ ]:
data_file_path = PROJECT_DIR + '/data/lstm/truyen_kieu_data_pre.txt'

def load_data() -> tuple:
    rs: map = {}
    ls = []
    tmp_line = ""
    with open(file=data_file_path, mode="rt") as i_f:
        count = 0
        for line in i_f:
            line = line.strip()
            if "" == line: continue
            if count % 2 == 0 and tmp_line!="":
                tokens = custokenizer.tokenize(tmp_line.strip())
                num_line = custokenizer.convert_tokens_to_ids(tokens=tokens)
                ls.append(num_line)
                # ls.append(tmp_line.strip())
                tmp_line = ""
                for num in num_line:
                    if num not in rs: rs[num] = 0
                    rs[num] += 1
                    pass
                pass
            tmp_line += " "+line
            count+=1
        pass
    return rs, ls

In [ ]:
bag_of_words, coupled_lines = load_data()
m = len(coupled_lines)
max_input_len = max(len(coupled_line) for coupled_line in coupled_lines)
print(f"max_input_len: {max_input_len}")
# bag_of_words["|"] = len(coupled_lines)
# bag_of_words["\\"] = len(coupled_lines) - 1
# ix_to_word = [*bag_of_words.keys()]
# keys = [*bag_of_words.keys()]
# word_to_ix = { w:i for i,w in enumerate(keys) }
# ix_to_word = { i:w for i,w in enumerate(keys) }
# coupled_lines
# values = [*bag_of_word.values()]
# sum(values)
# len(bag_of_words)
# ix_to_word
total_word = len(bag_of_words)
max_word_num = max(k for k in bag_of_words)

X_train = np.zeros(shape=(m, max_input_len,))
for i in range(m):
    line = coupled_lines[i]
    line_ids = custokenizer.convert_tokens_to_ids(line)
    for j in range(len(line_ids)):
        X_train[i, j] = line_ids[j]
        pass
    pass
print(f"X_train.shape:{X_train.shape}")
delta_line = 1
# y_train = [line[5] for line in X_train[delta_line:]]
y_train = [*X_train[delta_line:]]
y_train = [e[0] for e in y_train]
for c in range(delta_line):
    y_train.append(X_train[c][0])
    pass



print(f"total_word:{total_word} max_word_num:{max_word_num}")
# max_input_len = 14
X_train = np.asarray(a=X_train, dtype=int)
y_train = np.asarray(y_train) #.reshape(X_train.shape[0], 1)
# print(y_train.shape)
y_train = to_categorical(y_train, num_classes=total_word, dtype=int)
print(f"y_train:{y_train.shape}")
# X_train.shape
# y_train.shape
# len(coupled_lines[0].split())
# coupled_lines

In [ ]:
model: tf.keras.Model = tf.keras.Sequential(name="LSTM-RNN")
# input = tf.keras.layers.Input(shape=(15))
# embedding_1 = tf.keras.layers.Embedding(input_dim=total_word, output_dim=2048, input_length=max_input_len)
embedding_1 = tf.keras.layers.Embedding(input_dim=total_word, output_dim=2048, input_length=max_input_len)
bidrect_1 = tf.keras.layers.Bidirectional(\
    layer=tf.keras.layers.LSTM(units=max_input_len*32, return_sequences=True, go_backwards=False),\
    backward_layer=tf.keras.layers.LSTM(units=max_input_len*24, return_sequences=True, go_backwards=True))
dropout = tf.keras.layers.Dropout(rate=0.4)
bidrect_2 = tf.keras.layers.Bidirectional(layer=tf.keras.layers.LSTM(units=max_input_len*16, return_sequences=True))
dropout = tf.keras.layers.Dropout(rate=0.6)
lstm = tf.keras.layers.LSTM(units=max_input_len*16)
# output_1 = tf.keras.layers.Dense(units=total_word, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-2))
# output_2 = tf.keras.layers.Dense(units=total_word, activation="softmax")
output_1 = tf.keras.layers.Dense(units=total_word, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-2))
output_2 = tf.keras.layers.Dense(units=total_word, activation="softmax")

# model.add(input)
model.add(embedding_1)
# model.add(bidrect_1)
# model.add(bidrect_2)
# model.add(dropout)
model.add(lstm)
model.add(dropout)
model.add(output_1)
model.add(output_2)

model.summary()


In [ ]:
tf.config.set_soft_device_placement(True) 
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# loss_fn = tf.keras.losses.sparse_categorical_crossentropy
loss_fn = tf.keras.losses.categorical_crossentropy
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=loss_fn,
#               metrics=["accuracy"])
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3),
              loss=loss_fn,
              metrics=["accuracy"])

In [ ]:
# strategy  = tf.distribute.get_strategy()
# if model is not None: del model
tf.keras.backend.clear_session()
with strategy.scope():
# with tf.device('/cpu:0'):
# with tf.device("/gpu:0"):
    model.fit(x=X_train, y=y_train, validation_split=0.1, epochs=160, batch_size=128, verbose=1)
#     model.fit(x=X_train, y=y_train, epochs=75, batch_size=64, verbose=1)
    pass

In [ ]:
model.save('/kaggle/working/kieu_story')

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display
download_file_name = 'kieu_story'
path = f"/kaggle/working/{download_file_name}"
zip_name = f"/kaggle/working/{download_file_name}.zip"
command = f"zip {zip_name} {path} -r"
result = subprocess.run(command, shell=True, capture_output=True, text=True)
if result.returncode != 0:
    print("Unable to run zip command!")
    print(result.stderr)
display(FileLink(f'{download_file_name}.zip'))


In [ ]:
import random as rd

num_of_word = 14 * 4
seed = "đàn bà dễ có mấy tay đời xưa mấy mặt đời này mấy gan"
# seed_arr = [ix_to_word[rd.randint(0, total_word)] for _ in range(14)]
# seed_arr[11] = seed_arr[5]
# seed = " ".join(seed_arr).strip()
seed_arr_tokens = custokenizer.tokenize(seed)
# seed_sequence = [word_to_ix[word] for word in seed_arr]
seed_arr_ids: list = custokenizer.convert_tokens_to_ids(seed_arr_tokens)
# if len(seed_arr_ids) < max_input_len: sedd_arr_ids.append(0)
for i in range(max_input_len - len(seed_arr_ids)):
    seed_arr_ids.append(0)
    pass
rs = ""
for i in range(num_of_word):
    if i % max_input_len == 0 and i != 0:
#         seed_sequence.append(" \\")
        rs += ("\n")
#         continue
    if i % max_input_len == 6:
#         seed_sequence.append(" | ")
        rs += ("\n")
#         continue
    prediction = model.predict(np.array([seed_arr_ids]), verbose="0")
    predicted_word_idx = np.argmax(prediction)
#     predicted_word = ix_to_word[predicted_word_idx]
#     predicted_word = 
    
    seed_arr_ids.append(predicted_word_idx)
    
    if len(seed_arr_ids) % 6 == 0:
        seed_arr_ids = seed_arr_ids[6:]
#     seed += " " + predicted_word
#     rs += " " + predicted_word
#     rs += " ".join(custokenizer.convert_ids_to_tokens(seed_arr_ids),)
    print(seed_arr_ids)
    print(custokenizer.convert_ids_to_tokens(seed_arr_ids))

print(rs)

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phoBERT = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)